# Lab_07: Classification


In [1]:
!wget https://github.com/dbdmg/data-science-lab/raw/master/datasets/free-spoken-digit.zip -O free_spoken_digit.zip

--2022-05-11 21:22:24--  https://github.com/dbdmg/data-science-lab/raw/master/datasets/free-spoken-digit.zip
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/free-spoken-digit.zip [following]
--2022-05-11 21:22:25--  https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/free-spoken-digit.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9315920 (8,9M) [application/zip]
Saving to: ‘free_spoken_digit.zip’

free_spoken_digit.z 100%[===================>]   8,88M  20,2MB/s    in 0,4s    

2022-05-11 21:22:27 (20,2 MB/s) - ‘free_spoken_digit.z